In [17]:
import pandas as pd
import numpy as np

In [18]:
#importing the terms csv data obtained from the internet and creating Dataframe
AI_Terms= pd.DataFrame(pd.read_csv("AI_term_popularity.csv"))
Chat_GPT_Terms= pd.DataFrame(pd.read_csv("Chat_gpt_interest.csv"))